# Steps
1- Realease the particles (2_PT_seseflux)
2- Read the particles for each month 
3- reduce to firs particles intersection to coastline, delete the ones that does not interact with coastline 
4- we make a dataframe that only include the particles first intersect with the shoreline for each month 
- another thing we need to add is to add a group_number to the particles in addition that group_id like 11, 12, 13


In [1]:
# Importing libraries
import warnings 
import os
import glob
import xarray as xr
import geopandas as gpd
import pandas as pd
import dask.dataframe as dd
import gc
from shapely.geometry import Point

# Enable Dask progress bar
from dask.diagnostics import ProgressBar

# Reading Coastal wetlands shapefiles

In [2]:
# read the coastal wetland shapefile
CW_path ='/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/Coastalwetland/hitshoreline'
CW_avg = os.path.join(CW_path, 'Wetland_connected_avg_inundation_NAD1983_shorelineinteraction_50m_ExportFeatures.shp')
#CW_low = os.path.join(CW_path, 'wetlands_connected_low_inundation_NAD1983_shorelineinteraction_50m_ExportFeatures.shp')
#CW_high = os.path.join(CW_path, 'wetlands_connected_high_inundation_NAD1983_shorelineinteraction_50m_ExportFeatures.shp')
#CW_surge = os.path.join(CW_path, 'wetlands_connected_surge_inundation_NAD1983_shorelineinteraction_50m_ExportFeatures.shp')

In [3]:
CW_avg = gpd.read_file(CW_avg)
# CW_low = gpd.read_file(CW_low)
# CW_high = gpd.read_file(CW_high)
# CW_surge = gpd.read_file(CW_surge)
# Combine all shapefiles into one GeoDataFrame
#combined_shapefiles = pd.concat([CW_avg])

In [4]:
# Define the path to the data directory
data_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output'
def sort_key(file):
    filename = os.path.basename(file)
    number = int(filename.split('__')[1].split('.')[0])
    return number
files = glob.glob(data_dir + "/updated_Fvcome_huron_estuary_2023_*.nc")
files.sort(key=sort_key)
    

In [8]:
ds = xr.open_dataset(files[0])
ds

<xarray.Dataset>
Dimensions:                   (time: 121, particles: 4950, dim_0: 4950)
Coordinates:
  * time                      (time) datetime64[ns] 2023-01-01 ... 2023-01-31
Dimensions without coordinates: particles, dim_0
Data variables:
    group_id                  (particles) int32 ...
    longitude                 (time, particles) float32 ...
    latitude                  (time, particles) float32 ...
    depth                     (time, particles) float32 ...
    host_fvcom                (time, particles) int32 ...
    error_status              (time, particles) int32 ...
    in_domain                 (time, particles) int32 ...
    is_beached                (time, particles) int32 ...
    land_boundary_encounters  (time, particles) int32 ...
    thetao                    (time, particles) float32 ...
    so                        (time, particles) float32 ...
    group_number              (dim_0) int64 ...
Attributes:
    title:    PyLag -- Plymouth Marine Laboratory

In [5]:
# Open multiple NetCDF datasets with chunks using xarray
# Suppress FutureWarnings related to pandas unique
warnings.filterwarnings("ignore", category=FutureWarning, module="xarray")

# Open multiple NetCDF datasets with chunks using xarray
datasets = xr.open_mfdataset(files, combine='by_coords', parallel=True)

# Select relevant variables: time, group_id, group_number, longitude, latitude
selected_vars = datasets[['time', 'group_id', 'group_number', 'longitude', 'latitude']]
# Stack across 'time' and 'particles', dropping unwanted dimensions
stacked_data = selected_vars.stack(particle_time=('time', 'particles')).drop_dims('dim_0', errors='ignore')
stacked_data['longitude'].values = stacked_data['longitude'].values - 360

In [6]:
# Convert stacked xarray data to Dask DataFrame
df = stacked_data.to_dask_dataframe()
# Optionally trigger computation and see the head of the DataFrame
print(df.head())

                           particle_time       time  particles  group_id  \
0  (Timestamp('2023-01-01 00:00:00'), 0) 2023-01-01          0         0   
1  (Timestamp('2023-01-01 00:00:00'), 1) 2023-01-01          1         0   
2  (Timestamp('2023-01-01 00:00:00'), 2) 2023-01-01          2         0   
3  (Timestamp('2023-01-01 00:00:00'), 3) 2023-01-01          3         0   
4  (Timestamp('2023-01-01 00:00:00'), 4) 2023-01-01          4         0   

   longitude   latitude  
0 -84.665833  45.755062  
1 -84.671844  45.749062  
2 -84.673828  45.749062  
3 -84.657837  45.747059  
4 -84.659851  45.747059  


In [7]:
# save the dataframe to a csv file
df.to_csv('/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv')

['/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/00.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/01.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/02.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/03.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/04.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/05.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/06.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/07.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/08.part',
 '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/particletracking_output.csv/09.part',
 '/home/ab

In [12]:
# convert df to geodataframe
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf = gdf.set_crs('EPSG:4326')

In [ ]:
print (gdf.head())

In [29]:
import os
import glob
import xarray as xr
import geopandas as gpd
import pandas as pd
import gc
from shapely.geometry import Point
from dask.diagnostics import ProgressBar

# Enable Dask progress bar
pbar = ProgressBar()
pbar.register()

# Define the path to the data directory
data_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output'

# Dictionary to map abbreviated month names to a month number
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05',
    'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

# Function to extract the year and month from the filename
def sort_key(file):
    filename = os.path.basename(file)
    parts = filename.split('_')
    year_str = parts[4]  # Extract the year, e.g., '2023'
    month_str = parts[5].strip().capitalize()  # Normalize the month abbreviation
    if month_str not in month_map:
        raise KeyError(f"Month '{month_str}' not found in month_map!")
    month_num = month_map[month_str]
    return pd.to_datetime(f"{year_str}-{month_num}-01", format='%Y-%m-%d')

# Get the list of NetCDF files and sort them by the extracted date
files = glob.glob(os.path.join(data_dir, "updated_Fvcome_huron_estuary_2023_*.nc"))
files.sort(key=sort_key)


In [ ]:

# Process NetCDF files for each month, efficiently handling memory
for file in files:
        # Extract the month from the filename
        date = sort_key(file)
        month_key = date.strftime('%Y-%m')  # e.g., '2023-07'
        print(f"Processing month: {month_key}")

        # Open the first NetCDF file and select necessary variables
        ds = xr.open_dataset(file)
        selected_vars = ds[['time','group_id','group_number','longitude', 'latitude']].stack(particle_time=('time', 'particles')).drop_dims('dim_0')
        selected_vars['time'] = pd.to_datetime(selected_vars['time'])
        selected_vars['time'] = selected_vars['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        # Convert the xarray Dataset to a Dask DataFrame
        df = selected_vars.to_dask_dataframe(['group_id','group_number','longitude', 'latitude'])
        print(df.head())


In [ ]:
selected_vars['time'] 

In [ ]:
  # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
            crs="EPSG:4326"  # Assuming WGS84 projection
        )

        # Spatial join to filter only intersecting points
        intersecting_gdf = gpd.sjoin(gdf, CW_avg[['geometry']], how='inner', predicate='intersects')

        # Save intersecting data
        if not intersecting_gdf.empty:
            output_path = os.path.join(data_dir, f"geodata_{month_key}.csv")
            if not os.path.exists(output_path):
                intersecting_gdf[['latitude', 'longitude', 'time', 'group_id', 'group_number']].to_csv(output_path, index=False)
            else:
                intersecting_gdf[['latitude', 'longitude', 'time', 'group_id', 'group_number']].to_csv(output_path, mode='a', header=False, index=False)

            print(f"Saved intersecting data for {month_key}")

        # Clear memory and trigger garbage collection
        del df, gdf, intersecting_gdf
        gc.collect()

        # Close the dataset after processing
        ds.close()

    except Exception as e:
        print(f"Error processing {file}: {e}")
    
    # Force memory cleanup after each file
    gc.collect()

print("Processing complete!")


In [5]:
ds = xr.open_dataset(files[0]) 

In [ ]:
ds

In [ ]:
df = ds.to_dataframe().reset_index()
df = df[['time', 'group_id', 'group_number', 'longitude', 'latitude']]

# Find intersection of particle tracking with coastal watersheds